In [11]:
import json
import numpy as np
import pandas as pd

# FastAPI
import uvicorn
import nest_asyncio

nest_asyncio.apply()
from typing import Literal, List
from pydantic import BaseModel
from fastapi import FastAPI, UploadFile, Form, Depends

# Models
import joblib
from tensorflow.keras import models

In [12]:
def load_random_forest_model():

    model = open('../trainedModels/RandomForest_Model.pkl', 'rb')
    return joblib.load(model)

In [14]:
def prepare_random_forest_data(departamento, municipio, tipoPerdida, grupoEtario, genero,
                               cabezaFamilia, homicidiosProporcion, ano, indrural, gini,
                                IPM, coca, o_acto_terror):
    """
    Function to prepare data for the model prediction.
    Transforms the salary level into dummy variables
    
    @params satisfaction_level : float 
    @params average_monthly_hours : int 
    @params salary_level : str 
    @returns: DataFrame with the prepared data
    """
    # tipoPerdida_ = {"Abandono": [0, 0, 0], "Ambos": [1, 0, 0], "Despojo": [0, 1, 0], "NoReporta": [0, 0, 1]}



    cols = [
        'departamento', 'municipio', 'tipoPerdida', 'grupoEtario', 'genero',
        'cabezaFamilia', 'homicidiosProporcion', 'ano', 'indrural', 'gini',
        'IPM', 'coca', 'o_acto_terror'
    ]

    data = [[
        departamento, municipio, tipoPerdida, grupoEtario, genero,
        cabezaFamilia, homicidiosProporcion, ano, indrural, gini, IPM, coca, o_acto_terror
    ]]

    return pd.DataFrame(columns=cols, data=data)

In [15]:
prepare_random_forest_data(1, 2, 2, 2, 1, 2, 0.5, 2, 0.3, 0.5, 0.5, 1, 43)

,departamento,municipio,tipoPerdida,grupoEtario,genero,cabezaFamilia,homicidiosProporcion,ano,indrural,gini,IPM,coca,o_acto_terror
0,1,2,2,2,1,2,0.5,2,0.3,0.5,0.5,1,43


In [16]:
def prepare_random_forest_response(prediction):
    """
    Function to prepare the Random Forest resignation
    response 
    
    @params: np array with the prediction
    @returns: Dict with the resignation predicition : "Yes"/"No"
    """
    decision = {0: 'Compensa', 3: 'No Restituye', 4: 'Restituye'}
    return {"Resignation prediction": decision[prediction[0]]}

#### Building API

In [17]:
# API port
port = 8050

# Proxy
proxy = ""

service_prefix = os.getenv('JUPYTERHUB_SERVICE_PREFIX')

api_url = f'http://localhost:{port}/'
# if in workspace, set the proxy server prefix path & the API url
if service_prefix:
    proxy = f'{service_prefix}proxy/{port}/'
    api_url = f'https://workspace.ds4a.com{proxy}'

print("API URL: ", api_url)

API URL:  http://localhost:8050/


In [18]:
fastapi_random_forest = FastAPI(root_path=proxy)

In [19]:
@fastapi_random_forest.get('/status/')
async def get_api_status():
    """
    GET method for getting the status of the API

    @returns: JSON of the API status
    """
    return {'status': 'Up & running'}

In [22]:
loaded_model = load_random_forest_model()


@fastapi_random_forest.post('/predict/')
async def random_forest_predict(departamento, municipio, tipoPerdida, grupoEtario, genero,
                               cabezaFamilia, homicidiosProporcion, ano, indrural, gini,
                                IPM, coca, o_acto_terror):
    """
    POST method for predicting if an employee is going to resign to his/her job based on
    the level of satisfaction, average monthly hours & salary level, 
    using a Random Forest model.
    
    @params satisfaction_level : float 
    @params average_monthly_hours : int 
    @params salary_level : str 
    @returns: JSON of the resignation prediction
    """

    prepared_data = prepare_random_forest_data(departamento, municipio, tipoPerdida, grupoEtario, genero,
                               cabezaFamilia, homicidiosProporcion, ano, indrural, gini,
                                IPM, coca, o_acto_terror)

    prediction = loaded_model.predict(prepared_data)

    return prepare_random_forest_response(prediction)

In [23]:
print(f'API Swagger URL: {api_url}docs')
uvicorn.run(fastapi_random_forest, port=port, host='0.0.0.0')

API Swagger URL: http://localhost:8050/docs


In [ ]:
'departamento', 'municipio', 'tipoPerdida', 'grupoEtario', 'genero',
'cabezaFamilia', 'homicidiosProporcion', 'ano', 'indrural', 'gini',
'IPM', 'coca', 'o_acto_terror'

In [24]:
## Pydantic Model
class ModelInput(BaseModel):
    """
    Class that defines the model input typing, restrictions and max/min values for validation
    """
    departamento: int = Form(
        description="Departamento en donde se dio la sentencia", ge=0, le=25)
    municipio: int = Form(
        description="Municipio en donde se dio la sentencia", ge=0, le=368)
    tipoPerdida: int = Form(
        description="Tipo de pérdida involucrada", ge=0, le=3)
    grupoEtario: int = Form(
        description="Grupo etario del/de la solicitante", ge=0, le=5)
    genero: int = Form(
        description="Género del/de la solicitante", ge=0, le=3)
    cabezaFamilia: int = Form(
        description="Se es cabeza de familia", ge=0, le=1)
    homicidiosProporcion: float = Form(
        description="Homicidios totales/Población municipal", ge=0, le=1)
    ano: int = Form(
        description="Año de expedición de la sentencia", ge=0, le=10) 
    indrural: float = Form(
        description="Pob Rural/Pob total municipal", ge=0, le=1)
    gini: float = Form(
        description="Gini municipal", ge=0, le=1)   
    IPM: float = Form(
        description="IPM municipal", ge=0, le=100)
    coca: int = Form(
        description="Presencia de coca en el municipo", ge=0, le=1)
    o_acto_terror: int = Form(
        description="Actos de terror ocurridos en el municipio")      
                   
    # salary_level: Literal["high", "low", "medium"]

In [25]:
# How to pass parameters from a pydantic model into a helper function
def test_parameters(departamento, municipio, tipoPerdida, grupoEtario, genero,
                    cabezaFamilia, homicidiosProporcion, ano, indrural, gini,
                    IPM, coca, o_acto_terror):
    print(departamento, municipio, tipoPerdida, grupoEtario, genero,
        cabezaFamilia, homicidiosProporcion, ano, indrural, gini,
        IPM, coca, o_acto_terror)


# First convert the pydantic object into a dictionary
parameters = {
    'departamento': 2,
    'municipio': 3,
    'tipoPerdida': 2,
    'grupoEtario': 3,
    'genero': 1,
    'cabezaFamilia': 1, 
    'homicidiosProporcion': 0.006, 
    'ano': 3, 
    'indrural': 0.55, 
    'gini': 0.45,
    'IPM': 34, 
    'coca': 1, 
    'o_acto_terror': 12
}

# Use the **kwargs sintax to pass a dict into a function
test_parameters(**parameters)

2 3 2 3 1 1 0.006 3 0.55 0.45 34 1 12


In [26]:
fastapi_validation = FastAPI(root_path=proxy)


@fastapi_validation.post('/predict/')
async def predict_validation(inputs: ModelInput = Depends()):
    """
    POST method for predicting if an employee is going to resign to his/her job based on
    the level of satisfaction, average monthly hours & salary level, 
    using a Random Forest model. Applies data validation
    
    @params inputs: Model input for prediction
    @returns: JSON of the resignation prediction
    """

    prepared_data = prepare_random_forest_data(**dict(inputs))

    prediction = loaded_model.predict(prepared_data)

    return prepare_random_forest_response(prediction)


print(f'API Swagger URL: {api_url}docs')
uvicorn.run(fastapi_validation, port=port, host='0.0.0.0')

API Swagger URL: http://localhost:8050/docs


#### Probando request a la data

In [27]:
import pandas as pd
import requests

In [ ]:
# multiple_data = pd.read_csv('')
# data = list(multiple_data[:2].to_dict(orient='index'.values()))
# data

In [32]:
data = [{
    "departamento": 2,
    "municipio": 3,
    "tipoPerdida": 2,
    "grupoEtario": 3,
    "genero": 1,
    "cabezaFamilia": 1, 
    "homicidiosProporcion": 0.006, 
    "ano": 3, 
    "indrural": 0.55, 
    "gini": 0.45,
    "IPM": 34, 
    "o_acto_terror": 12
}] 
url = 'http://localhost:8050/predict/'
response = requests.post(url, json=data)
response.json()

[{'Resignation prediction': 'Restituye'}]

In [33]:
pd.DataFrame(response.json())

,Resignation prediction
0,Restituye
